In [8]:
from rich import print
import os 
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("GITHUB_TOKEN")
if openai_api_key is None:
    print("Please set OPENAI_API_KEY in .env file")

In [9]:
import requests

def get_weather(latitude: str, longitude: str):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [10]:
from openai import OpenAI

client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=openai_api_key
)

# {"latitude":48.8566,"longitude":2.3522}

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print(completion)

ChatCompletion(
    id='chatcmpl-BCphNNx3qvaRGIQ2r7DhMUyHawKTj',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                refusal=None,
                role='assistant',
                annotations=None,
                audio=None,
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_PMIn4iW69Of3zNlf9DJnzG2p',
                        function=Function(
                            arguments='{"latitude":48.8566,"longitude":2.3522}',
                            name='get_weather'
                        ),
                        type='function'
                    )
                ]
            ),
            content_filter_results={}
        )
    ],
    created=1742398365,
    model='gpt-4o-2024-11-20',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_ded0d14823',
    usage=CompletionUsage(
        completion_tokens=25,
        prompt_tokens=60,
        total_tokens=85,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    ),
    prompt_filter_results=[
        {
            'prompt_index': 0,
            'content_filter_results': {
                'hate': {'filtered': False, 'severity': 'safe'},
                'jailbreak': {'filtered': False, 'detected': False},
                'self_harm': {'filtered': False, 'severity': 'safe'},
                'sexual': {'filtered': False, 'severity': 'safe'},
                'violence': {'filtered': False, 'severity': 'safe'}
            }
        }
    ]
)

In [11]:
import json
tool_call = completion.choices[0].message.tool_calls[0]
print(tool_call)
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])
print(result)

ChatCompletionMessageToolCall(
    id='call_PMIn4iW69Of3zNlf9DJnzG2p',
    function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'),
    type='function'
)

18.2

In [12]:
messages = []

messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    )

In [13]:
print(completion_2)

ChatCompletion(
    id='chatcmpl-BCpi7kiRSXshljfRKguAFlaIkklRf',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='The current temperature in Paris is 18.2°C.',
                refusal=None,
                role='assistant',
                annotations=None,
                audio=None,
                function_call=None,
                tool_calls=None
            ),
            content_filter_results={
                'hate': {'filtered': False, 'severity': 'safe'},
                'protected_material_code': {'filtered': False, 'detected': False},
                'protected_material_text': {'filtered': False, 'detected': False},
                'self_harm': {'filtered': False, 'severity': 'safe'},
                'sexual': {'filtered': False, 'severity': 'safe'},
                'violence': {'filtered': False, 'severity': 'safe'}
            }
        )
    ],
    created=1742398411,
    model='gpt-4o-2024-11-20',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_ded0d14823',
    usage=CompletionUsage(
        completion_tokens=14,
        prompt_tokens=82,
        total_tokens=96,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    ),
    prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {}}]
)